In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
print(os.listdir("/kaggle/input/fruits/fruits-360_dataset/fruits-360"))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [9]:
import tensorflow as tf
from tensorflow.keras import backend, models, layers, optimizers
import numpy as np
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import plot_model
from IPython.display import display
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os, shutil
from tensorflow.keras.models import Model
np.random.seed(42)

In [10]:
base_dir = "/kaggle/input/fruits/fruits-360_dataset/fruits-360/"
train_dir = os.path.join(base_dir, 'Training')
test_dir = os.path.join(base_dir, 'Test')

In [12]:
train_data = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir, image_size=(224,224), batch_size=32, # we can try to alter this on GPU
    label_mode="categorical"
)
test_data = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir, image_size=(224,224), batch_size=32, # we can try to alter this on GPU
    label_mode="categorical"
)

In [14]:
from tensorflow.keras.layers.experimental import preprocessing
data_augmentation = tf.keras.Sequential([
    preprocessing.RandomFlip("horizontal"),
    preprocessing.RandomZoom(0.4),
    preprocessing.Rescaling(1/255.)
], name="data_augmentation")

In [15]:
input_shape=(224,224,3)

In [17]:
# Making model using functional API
# I am also going to use Inception-V3
from tensorflow.keras.applications.inception_v3 import InceptionV3
backend.clear_session() # Helps to avoid cluttering of memory, releases global state

In [19]:
# Base model
conv_base = InceptionV3(weights = 'imagenet', #Useing the inception_v3 CNN that was trained on ImageNet data.  
                  include_top = False)

inputs = tf.keras.Input(shape=input_shape,name="input_layer")
x = data_augmentation(inputs)
x = conv_base(x,training=False)
x = GlobalAveragePooling2D()(x)
x = layers.Dense(512, activation = 'relu')(x)
outputs = layers.Dense(131,activation="softmax")(x)
base_model_to_view = Model(inputs,outputs)

In [20]:
plot_model(base_model_to_view)

In [22]:
model_InceptionV3 = models.Model(inputs=inputs, outputs=outputs)
# Compile the model with categorical crossentropy for the loss function and SGD for the optimizer with the learning
# rate at 1e-4 and momentum at 0.9
model_InceptionV3.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(learning_rate=1e-4, momentum=0.9),
              metrics=['accuracy'])
# Usually in transfer learning, we decrease the learning rate by 10 times
# Momentum alters the gradient descent update rule

In [23]:
# Check GPU
!nvidia-smi

In [25]:
hist_incep = model_InceptionV3.fit(
        train_data,
        epochs=5,
        validation_data=test_data,
        steps_per_epoch = len(train_data),
        validation_steps = int(0.2*len(test_data)),
        callbacks=[EarlyStopping(monitor='val_accuracy', patience = 5, restore_best_weights = True)])

In [34]:
import pandas as pd
pd.DataFrame(hist_incep.history).plot()

In [105]:
# urllib.request.urlretrieve(plum,"plum")
  
# i = Image.open("plum")
# ri = i.resize((224,224))
# arr = np.array(ri)
# print(predict(model_InceptionV3,arr/255.))